## 0. Getting started

In [1]:
import scipy
import scipy.io.wavfile
from scipy import signal
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob

#import cmath
#import math

import librosa

from collections import defaultdict

import json

import copy

import sys

/home/c/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# Getting the used versions of imports:
import matplotlib, prettytable 
import_list = [sys, scipy,np,pd,librosa,json] # sys gives us the python version
my_versions = ['3.7.3','1.4.1 ','1.19.2 ','1.1.4','0.9.1','2.0.9','']
for ele, my_version in zip(import_list, my_versions):
    try:
        v = ele.__version__
        print
    except:
        try:
            v = ele.version
            
        except:
            v = 'cant say version'
    print(ele, ': \nyour version ', v, '\noriginally used version ', my_version, '\n')

<module 'sys' (built-in)> : 
your version  3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0] 
originally used version  3.7.3 

<module 'scipy' from '/home/c/anaconda3/lib/python3.7/site-packages/scipy/__init__.py'> : 
your version  1.4.1 
originally used version  1.4.1  

<module 'numpy' from '/home/c/anaconda3/lib/python3.7/site-packages/numpy/__init__.py'> : 
your version  1.19.2 
originally used version  1.19.2  

<module 'pandas' from '/home/c/anaconda3/lib/python3.7/site-packages/pandas/__init__.py'> : 
your version  1.1.4 
originally used version  1.1.4 

<module 'librosa' from '/home/c/.local/lib/python3.7/site-packages/librosa/__init__.py'> : 
your version  0.9.1 
originally used version  0.9.1 

<module 'json' from '/home/c/anaconda3/lib/python3.7/json/__init__.py'> : 
your version  2.0.9 
originally used version  2.0.9 



In [3]:
def autocorrelation(array):
    stopping_t = len(array)#-60
    if stopping_t < 0:
        raise Exception('Inputted array is too small!')
        return None
    
    d1 = np.copy(array)[:stopping_t]
    d2 = np.copy(array)[:len(d1)]
   
    r = [] 

    for tau in range(0,len(array)):
             
        summed_multiplication = np.sum(d1*d2)
        d1 = d1[1:]
        d2 = d2[:-1]

        r.append(summed_multiplication)

        
    return r

In [4]:
def crosscorrelation(array1, array2):
    
    # arrays have same length
    
    stopping_t = array1.shape[1]#-60
    if stopping_t < 0:
        raise Exception('Inputted array is too small!')
        return None
    
    d1 = np.copy(array1)[:,:stopping_t]#60:stopping_t
    d2 = np.copy(array2)[:,:d1.shape[1]]

    first_round = True
    for tau in range(0,array1.shape[1]):#60,201):#(0,len(d1)):#(61,201):
        

        summed_multiplication = np.sum(d1*d2,axis=1).reshape((d1.shape[0],1))
        d1 = d1[:,1:]
        d2 = d2[:,:-1]
        # stopping_t -= 1 # every time tau grows we come closer to the final index of the array. Therefore, 
        # t has to stop earlier.
        if first_round == True:
            first_round = False
            r = summed_multiplication
        else:
            r = np.concatenate((r, summed_multiplication),axis=1)

        
    return r

In [5]:
def picking_peaks(detection_func_norm_thres, max_ind_value, mean_ind_value, distance, delta):
    # the function is a bit more complicated than simply looping over the whole
    # onset detection function. With the array 'all_peaks' we want to look only at 
    # values which are bigger than 0 in the onset detection function. Furthermore,
    # we not simply loop over the 'all_peaks function', also here we make jumps when
    # a) through a found onset the minimal distance for the next potential onset has
    # to be considered and b) when we have within one window for which we search the 
    # maximal value we have several indices which fulfill the maxima condition. 
    # Therefore, we would jump to the next maximum with same value when the first was
    # not able to be a onset because it could not top the mean of the mean window.
    
    all_peaks = np.where(detection_func_norm_thres>0)[0]
   
    
    start_searching = None
    peak_collector = [] # will be the final list with all onset ind values

    # condition 1: has to be a max in a certain window
    single_peak_ind = 0
    curr_peak_ind = all_peaks[single_peak_ind]
    # single_peak_ind: ind in the all_peaks where every value is bigger than 0
    # curr_peak_ind: where the bigger-than-zero peak is within the onset detection function

    while single_peak_ind < len(all_peaks):

        lower_max_ind = (curr_peak_ind-max_ind_value) 
        if lower_max_ind < 0 or False:
            lower_max_ind = 0

        if start_searching is not None:
            start_searching = None
            lower_max_ind = single_peak_ind

        upper_max_ind = (curr_peak_ind+max_ind_value) 
        if upper_max_ind >= len(detection_func_norm_thres) or False:
            upper_max_ind = len(detection_func_norm_thres)-1
            

        max_window = np.array([lower_max_ind,upper_max_ind])
        max_windowed_func = detection_func_norm_thres[lower_max_ind:upper_max_ind+1]
        found_max_ind = np.argmax(max_windowed_func)
        found_max_value = max_windowed_func[found_max_ind]

        # find out if there is a second max with the same value:
        copied = np.copy(max_windowed_func)
        copied[found_max_ind] = 0
        found_same_max_ind = np.where(copied == found_max_value)[0]

        # condition 2: is the max bigger than the mean of a second surrounding window?
        lower_mean_ind = (curr_peak_ind-mean_ind_value) 
        if lower_mean_ind < 0 or False:
            lower_mean_ind = 0

        upper_mean_ind = (curr_peak_ind+mean_ind_value) 
        if upper_mean_ind >= len(detection_func_norm_thres) or False:
            upper_mean_ind = len(detection_func_norm_thres)-1

        mean_window = np.array([lower_mean_ind,upper_mean_ind])
        found_mean_value = np.mean(detection_func_norm_thres[lower_mean_ind:upper_mean_ind+1]) + delta
     
        if found_max_value >= found_mean_value: # we found an onset
            peak_collector.append(curr_peak_ind)

            # condition 3: jump to the next point which is fulfilling the distance measurement
            if (single_peak_ind+distance) < len(detection_func_norm_thres):
                est = curr_peak_ind + distance
                try:
                    single_peak_ind = np.where(all_peaks>=est)[0][0] # which 
                    # peak value bigger 0 comes closest to the minimal distance
                    curr_peak_ind = all_peaks[single_peak_ind]
                except: # single_peak_ind = np.where(all_peaks>=est)[0] = []
                    break
            else:
                break

        else: # we did not found an onset
            # the next max_window shall start at the index after the maximal value of the 
            # last used max_window, else the next value would not be able to be bigger than the 
            # last max

            # special case: we have found several maxima with the same value in one max_window.
            # Therefore, we jump to them for the next loop. When the mean value lowers, they 
            # have the chance to be taken as peak:
            if found_same_max_ind != []:
                #single_peak_ind 
                curr_peak_ind = lower_max_ind + found_same_max_ind[0]
                single_peak_ind = np.where(all_peaks==curr_peak_ind)[0][0]
                start_searching = not None # can be anything

            else:
                # even for values which are not max within the window:
                # when the max value vanishes with the moving max_window, they can become the max.
                # Second opportunity is that the mean value of the shifted mean_window lowers.
                single_peak_ind += 1# upper_max_ind 
                try:
                    curr_peak_ind = all_peaks[single_peak_ind]
                except: # we had a look at all peaks bigger than 0
                    break
    return np.asarray(peak_collector)

In [6]:
def mel_filterbank(sample_rate, hop_size, segment_times, STFT):

    # from http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/#eqn1
    # and https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html

    STFT11 = np.copy(STFT)

    # 0. finding out the sample rate for one window:
    len_window = int(segment_times[1])
    in_sec_window = (len_window-1) * hop_size/ sample_rate
    sample_rate2 = int(in_sec_window * sample_rate) # that's the number of samples in one window


    # 1. lower border of human hearing https://www.audiologyresearch.org/human-hearing-range: 
    # 20 Hz:
    # get borders mel band:
    variable = 2959 # values seen in lecture and internet so far: 2959, 1125, 2595
    
    mel_lowest_f = (variable * np.log10(1 + (20) / 700))
    mel_highest_f = (variable * np.log10(1 + (sample_rate2 / 2) / 700))  # Convert Hz to Mel

    # 2. creating 40 bins/bands from which 38 are equally distributed between the lowest and
    # highest mel value:
    number_filters = 20 #40 # normal: 20-40, standard: 26
    # with 20 no final_bins with same value and in filterbank no row with set={0.0}

    distance_mel = (mel_lowest_f + mel_highest_f)/(number_filters+1)
    mel_binDistances = np.array([mel_lowest_f + i * distance_mel for i in range(0, number_filters+2)])

    # 3. Converting the the mel_bins into frequencies:
    f_binDistances = (700 * (10**(mel_binDistances / variable) - 1)) #700*(np.exp(mel_binDistances/2959)-1)
    # because log_10(b)=x leads to 10**x=b

    # 4. Missing freq. resolution. Therefore, round the found freq. to nearest DFT bin.
    # Need of number of DFT bins and sample_rate:
    final_bins = np.floor((STFT.shape[0]+1)*f_binDistances/sample_rate2) # last bin relates
    # to mel_highest_f

    # 5. Create filterbanks: 
    # Each filterbank starts at point i, has the peak at i+1 and is zero at i+2.
    # formula H_m(k) from http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/
    # where k=...-th_filter, f=final_bins. Sentence before describes formula
    filterbank = np.zeros((number_filters,int(np.floor(STFT.shape[0] ))))
    # rows: number used filterbanks/individual filters, columns: values for certain frequencies

    for m in range(1,(number_filters+1)):
        left_val = int(final_bins[m-1])
        middle_val = int(final_bins[m])
        right_val = int(final_bins[m+1])

        for k in range(left_val, middle_val): # number of fitlers m between final bins
            filterbank[m-1,k] = (k-final_bins[m-1])/(final_bins[m]-final_bins[m-1])
        for k in range(middle_val, right_val):
            filterbank[m-1,k] = (final_bins[m+1]-k)/(final_bins[m+1]-final_bins[m])

    # 6. lecture content L04 slide 54: the magnitude approach:
    # Use the filterbank for each DFT magnitude to reduce it in bin size:
    
    shape_border = int(STFT.shape[1]/2)
    S1 = (filterbank @ abs(STFT[:,:shape_border]))
    S2 = (filterbank @ abs(STFT[:,:STFT.shape[1]]))
    S = np.concatenate((S1,S2),axis=1)
    S = 10 * np.log10(S+1) # rows: number of filters, column: like in STFT the 


    return S

In [7]:
def onset_detection(file):    

    sample_rate, data = scipy.io.wavfile.read(file)
    
    
    resampled = data
    #resampled = signal.resample(data, num=50000, t=None, axis=0, window=None)
    number_overlap = 100
    sample_freq, segment_times, STFT = signal.stft(resampled,noverlap=number_overlap) # all have the same size already!
    # noverlap: the number of samples two neighbouring DFT have in common / reverse of hop size
     
    hop_size = int(segment_times[1] - number_overlap)
    
    STFT = mel_filterbank(sample_rate, hop_size, segment_times, STFT)
    # of course its the filtered STFT
    
    """ COMPLEX DOMAIN
    # Create complex domain:
    angle_matrix = np.angle(STFT) # in radians not degrees (algebraic angle here)
    exponentialized_matrix1 =np.exp(1j*angle_matrix)

    delta_matrix =np.zeros(angle_matrix.shape)
    matrix = angle_matrix # phase_matrix
    for time in reversed(range(matrix.shape[1])):
       # print(np.all(out_matrix[:,-1]==0))

        if time==2:
            delta_matrix[:, time-1] = (matrix[:,time-1] - matrix[:,time-2]) #- matrix[time-2]

        elif time==1:
            delta_matrix[:,time-1] = matrix[:,0] # or 0 ???

        elif time==0:
            break
        else:
            delta_matrix[:, time-1] = (matrix[:,time-1] - matrix[:,time-2]) - (matrix[:,time-2] - matrix[:,time-3])

    exponentialized_matrix2 = np.exp(1j*delta_matrix)
    """
    
    abs_STFT = np.abs(STFT)
    
    """ COMPLEX DOMAIN
    STFT_prediction = abs_STFT * exponentialized_matrix1 / exponentialized_matrix2
    
    substractor_matrix = np.abs(STFT - STFT_prediction)
    detection_func = np.sum(substractor_matrix, axis=0) # get for every column/DFT one value
    """
    
    spec_diff =  abs_STFT[:,1:] - abs_STFT[:,:-1]
    spec_diff[np.where(spec_diff<0)] = 0
    detection_func = np.sum(spec_diff, axis=0)
 
    # Postprocess detection function:
    
    # Normalize detection function:
    mean = detection_func.mean()
    std = detection_func.std()
    detection_func_norm = (detection_func-mean)/std

    # Adaptive thresholding:
    delta_ = 0 #0.25#0.5 # 3#1 #0
    lambda_ = 1
    window_size = 25 # shall be odd number!

    threshold_array = np.array([])

    for curr_ind, curr_pt in enumerate(detection_func_norm):

        # getting window taking the index edges into account: 
        if window_size % 2 == 0: # if it is even
            upper = int((window_size)/2) # window is on right sight one step longer
            lower = int((window_size)/2)-1
        else:
            upper = int((window_size-1)/2)
            lower = int((window_size-1)/2)

        lower_window_ind = max(0, curr_ind-lower)
        upper_window_ind = min(len(detection_func_norm)-1, curr_ind+upper)



        window = detection_func_norm[lower_window_ind:upper_window_ind+1]
        window_size_sorted = np.sort(window)

        # getting median value and index:

        if window_size % 2 == 0: # if it is even

            lower_ind = int((window_size)/2-1)
            upper_ind = lower_ind+1

            median = (window_size_sorted[lower_ind] + window_size_sorted[upper_ind])/2
        else:
            median = window_size_sorted[int((window_size-1)/2)]
            
            

        # comparison values:    
        threshold_array = np.append(threshold_array, median)
    threshold_array = lambda_ * threshold_array + delta_
      
#     plt.plot([i for i in range(len(detection_func))],detection_func, c='green')
#     plt.show()
#     plt.plot([i for i in range(len(detection_func_norm))],detection_func_norm, c='red')
#     plt.plot([i for i in range(len(threshold_array))],threshold_array,'.', c='blue')
#     plt.show()
    detection_func_norm_thres = np.copy(detection_func_norm)
  
    try:
        detection_func_norm_thres[np.where(threshold_array>detection_func_norm_thres)[0]] =  min(detection_func_norm_thres)# 0
    except:
        print('no thres')
        pass
 
    # Getting onsets:
    
    
    # Peak picking:
    max_window_half_size, mean_window_half_size, distance = 14, 18, 48

    indices = picking_peaks(detection_func_norm_thres, max_window_half_size, mean_window_half_size, distance, delta_)
  
    
    predicted_onset_arr_time = list(indices * hop_size / sample_rate) # convertion
    
    return detection_func_norm_thres, predicted_onset_arr_time, sample_rate, hop_size

In [8]:
def tempo_detection(detection_func_norm_thres,splitter,hop_size,sample_rate):
    
    # Tempo estimation:
    
    # autocorrelation:
    #full_time = librosa.get_duration(data)
    
    # Compute place in detection autocorrelation function where tempo is 60 and 200 bpm:
    place_60 = 60 * sample_rate / (splitter* 60 * hop_size)
    place_200 = 60 * sample_rate / (splitter*200 * hop_size)
    lower_ind = int(place_200)+1
    upper_ind = int(place_60)
    
    # Splitting the detection function into windows such that we can also consider different tempi in only one
    # piece:
    window_len = int(len(detection_func_norm_thres)/splitter)
    window_ind = np.array([i*window_len-1 for i in range(1,splitter+1)])
    last_ind = 0
    tempo_list = []
    indicating_list = []
    start = True
    inder = []
    
    '''full autocorrelation without windows:
    place_60 = 60 * sample_rate / ( 60 * hop_size)
    place_200 = 60 * sample_rate / (200 * hop_size)
    lower_ind = int(place_200)+1
    upper_ind = int(place_60)
    r = autocorrelation(detection_func_norm_thres)    
    r2 = r[lower_ind:upper_ind+1]
    r2 = r2[1:]
    tempo_ind = np.argmax(r2)#+1
    tempo_full_ind = lower_ind + tempo_ind

    tempo_found =   60 * sample_rate / ((tempo_full_ind+1) * hop_size)
    
    r2.remove(r2[tempo_ind])
    tempo_ind2 = np.argmax(r2)
    tempo_full_ind2 = lower_ind + tempo_ind2
    tempo_found2 =   60 * sample_rate / ( (tempo_full_ind2+1) * hop_size)
    return [tempo_found, tempo_found2],[tempo_found,tempo_found2], 0, detection_func_norm_thres, 0
    #return true_tempi, tempi_window_sec, start_window_sec, detection_window_,start_window_ind
    '''
    # autocorrlation windowed:
    
    tempi_window_sec, start_window_sec, start_window_ind ,detection_window_ = [], [],[], None


    for ind in window_ind:
        detection_window = detection_func_norm_thres[last_ind:ind+1]
        lower_border_sec =  last_ind * hop_size / sample_rate#60 * sample_rate / (splitter* last_ind * hop_size)

        start_window_sec.append(lower_border_sec)  
       # start_window_sec.append(lower_border_sec) 
        
        start_window_ind.append(last_ind)  
       # start_window_ind.append(last_ind) 
        
        ind_saved = last_ind
        last_ind = ind + 1
        r_windowed = autocorrelation(detection_window)#detection_func_norm_thres) # r gives us the raw autocorrelation of d[t]
        r_windowed_cutted = r_windowed[lower_ind:upper_ind+1]
        tempo_ind = np.argmax(r_windowed_cutted)
        
        r_windowed_cutted[tempo_ind] = -1
        tempo_ind2 = np.argmax(r_windowed_cutted) # get second highest value

        tempo_full_ind = lower_ind + tempo_ind 
        tempo_found =  60 * sample_rate / (splitter* tempo_full_ind * hop_size)
        tempo_list.append(tempo_found)
        indicating_list.append(r_windowed_cutted[tempo_ind2])

        tempo_full_ind2 = lower_ind + tempo_ind2
        tempo_found2 = 60 * sample_rate / (splitter* tempo_full_ind2 * hop_size)
        tempo_list.append(tempo_found2)
        if tempo_found2 > tempo_found:
            insert = [tempo_found,tempo_found2]
        else:
            insert = [tempo_found2,tempo_found]
        tempo_array_part = np.array(insert).reshape((1,2))
        if start == True:
            start = False
            tempo_array = tempo_array_part
        else:
            tempo_array = np.concatenate([tempo_array, tempo_array_part],axis=0)

        tempo_window_sec = tempo_full_ind * hop_size / sample_rate #60 * sample_rate / (splitter* tempo_full_ind * hop_size) # assuming window starts at t=0
        tempo_window_sec2 = tempo_full_ind2 * hop_size / sample_rate#60 * sample_rate / (splitter* tempo_full_ind2 * hop_size)
        tempi_window_sec.append(tempo_window_sec)
        #tempi_window_sec.append(tempo_window_sec2)



        if detection_window_ is None:
            detection_window_ = detection_window.reshape(1,len(detection_window))
        else:
            detection_window_ = np.concatenate((detection_window_,detection_window.reshape(1,len(detection_window))),axis=0)


        inder.append(tempo_full_ind+ind_saved) # to not only look at window of detection function but also 
        # to full detection function. where is the tempo index in there?
        inder.append(tempo_full_ind2+ind_saved)


    # Taking median:
    tempo_array = np.sort(tempo_array, axis=0)
    tempo_array2 = np.copy(tempo_array)
    if tempo_array.shape[0] %2 == 0: # even number of rows
        upper_middle = int(tempo_array.shape[0] /2 )# upper one because we want index
        lower_middle = upper_middle -1 
        median_value = (tempo_array[upper_middle,:]+tempo_array[lower_middle,:])/2

    else:
        middle = int(tempo_array.shape[1] /2)
        median_value = tempo_array[middle,:]

    #dd[title]['tempo'] = median_value.tolist()
    
    tempo1 = max(tempo_list)

    tempo2 = min(tempo_list)

    where_max = tempo_list.index(tempo1)
    where_min = tempo_list.index(tempo2)
    detection_func_tempo1 = inder[where_max]
    detection_func_tempo2 = inder[where_min]
    """
    # take two tempi with highest indications:
    tempo_value1 = tempo_list[np.argmax(indicating_list)]
    tempo_list.remove(tempo_value1)
    indicating_list.remove(max(indicating_list))
    tempo_value2 = tempo_list[np.argmax(indicating_list)]

    if tempo_value1 > tempo_value2:
        sol = [tempo_value2, tempo_value1]
    else:
        sol = [tempo_value1, tempo_value2] 

    meaning_sol = np.mean(tempo_array2,axis=0)     

    if tempo_found > tempo_f
    tempo_found]
    else:
        sol = [tempo_found, tempo_found2] 
    """  
    true_tempi = [tempo1]#[tempo2, tempo1]

    return true_tempi, tempi_window_sec, start_window_sec, detection_window_,start_window_ind

In [9]:
def beat_detection(start_window_ind, start_window_sec, tempo_list_sec, detection_func_norm_thres_wind, hop_size, splitter, sample_rate, predicted_onset_arr_time):
    
    
    onset_func_sec = detection_func_norm_thres_wind #* hop_size / sample_rate #60 * sample_rate / (splitter* detection_func_norm_thres_wind * hop_size)
    indd = np.array([i for i in range(onset_func_sec.shape[1])])
    onset_function_x_axis = indd #list(indd * hop_size / sample_rate) 
  
    # 1. compute for each windowed tempo a sinus/cosine/pulse train 
    # by the shape=(number tempi, single function values):
    
    onset_function_x_axis_rep = np.array(start_window_ind).reshape((len(start_window_ind),1)) + np.vstack(([onset_function_x_axis for i in range(len(tempo_list_sec))]))
                                            
    cos2 = lambda x: abs(np.cos(2*np.pi*np.asarray(tempo_list_sec).reshape((len(tempo_list_sec),1)) * np.array(x)))
   
    # We want that the cos has its first peak at the period=1/tempo, therefore we use cos(B*x)
    # with the formula period = 2 pi/ abs(B) such that abs(B) = 2 pi tempo.
    cos2_sampled = cos2(onset_function_x_axis_rep)

    # 2. Cross-correlation between computed function from 1. and detection window recomputed in seconds
    # (each starting at second=0):
    onset_sec_lifted = onset_func_sec  + abs(min(onset_func_sec.flatten() )) # lowest y-value of function shall be 0
    r_cross2 = crosscorrelation(cos2_sampled,onset_sec_lifted)

    f1 = np.asarray([[1, 0, 1]])
    valley_inds = np.where(r_cross2 < scipy.ndimage.minimum_filter(r_cross2, footprint=f1, mode='reflect', cval=-np.inf))
    # fancier method than nested for loop. Compare for each line where one value is smaller than both of its neighbours
    # compare each value with horizontal neighbours. When it is smaller than both we have a valley. We need that
    # because we want to ignore the first hill where cross-correlation is of course very high 
    # (correlating two full arrays with each other). 
    
    r_cross2_copy = np.copy(r_cross2)
    if len(valley_inds[0]) > 0:
        
        valley_inds_rows, ind_val = np.unique(valley_inds[0], return_index=True)
        valley_inds_col = valley_inds[1][ind_val]

        for row_ind in range(r_cross2.shape[0]):
            curr_col = valley_inds_col[row_ind]
            r_cross2_copy[row_ind][:curr_col+1] = -9999
    else:
        r_cross2_copy[:,0] = -9999

    
    beat_indices_part = np.argmax(r_cross2_copy,axis=1) # getting for each line the first maximal value, here our first beat is located
    # getting beat for each time window beat estimation
    # 1D array!
    # in ind of seconds function!
    # we have small cross-correlation window in which we search for our max with ignoring the first hill. 
    # Here we get the final ind of first beat for EACH cross-correlations (which number is the number of estimated tempi)

    
    beat_indices =  beat_indices_part #np.asarray(start_window_ind) + beat_indices_part
    beat_ind = beat_indices
    
    # 3. Compute the following beat values in seconds within the single onset detection function windows:
    next_beat_indices = beat_indices # 1D array!
    beat_collector_sec = [] # 4. collect all estimated beats in seconds
    number_beats = len(beat_indices)
    #onset_function_x_axis_rep = np.vstack(([(onset_function_x_axis for i in range(number_beats)]))
    
    while np.any(next_beat_indices) < onset_func_sec.shape[1]: #*splitter:
        #next_beat_sec = onset_function_x_axis_rep[valley_inds_rows, next_beat_indices]
        
        forbidden_ind = set(np.where(next_beat_indices>=onset_function_x_axis_rep.shape[1])[0])
       
        if len(forbidden_ind) > 0:
            
            if len(forbidden_ind) == len(next_beat_indices):
                break
            
            full_ind_arr = set([i for i in range(len(next_beat_indices))])
            
            allowed_ind = np.asarray(list(full_ind_arr - forbidden_ind))
            
            
            next_beat_indices = next_beat_indices[allowed_ind]
            beat_ind = beat_ind[allowed_ind]
            start_window_ind = np.asarray(start_window_ind)[allowed_ind]
            number_beats -= len(forbidden_ind )
        
        next_beat_sec = (start_window_ind + next_beat_indices) * hop_size / sample_rate 
        
        if len(next_beat_sec) == 0:
            break
        
        #closest_onsets_ind = np.where(((np.asarray(predicted_onset_arr_time)>=next_beat_sec) & (np.asarray(predicted_onset_arr_time)<=next_beat_sec+0.05)) | ((np.asarray(predicted_onset_arr_time)<=next_beat_sec) & (np.asarray(predicted_onset_arr_time)>=next_beat_sec-0.05)))
        for i in range(number_beats):
            closest_onsets_ind = np.where(((np.asarray(predicted_onset_arr_time)>=next_beat_sec[i]) & (np.asarray(predicted_onset_arr_time)<=next_beat_sec[i]+0.05)) | ((np.asarray(predicted_onset_arr_time)<=next_beat_sec[i]) & (np.asarray(predicted_onset_arr_time)>=next_beat_sec[i]-0.05)))
            
            # find the onset which is closest to our beat location estimation:
            try:
                closest_onset_ind_ind = np.argmin(abs(np.asarray(predicted_onset_arr_time)[closest_onsets_ind]-next_beat_sec[i]))
                closest_onset_ind = closest_onsets_ind[0][closest_onset_ind_ind]
                closest_onset_sec = predicted_onset_arr_time[closest_onset_ind]
            except: # the closest_onsets_ind is empty. We have no onsets that are close enough to our beat estimation point. 
                # Therefore, we take the beat estimation point directly as next beat assumption.
                closest_onset_sec = next_beat_sec[i]
                        
            beat_collector_sec.append(closest_onset_sec )
        
        next_beat_indices += beat_ind
            
        
    return list(np.sort(beat_collector_sec))

In [10]:
path = "TestData/test/"
#path = "TrainData/train/"

filenames = glob.glob(path + "*.wav") # read all the files with extension .wav
label_filenames = glob.glob(path + '*.onsets.gt')
tempi_filenames = glob.glob(path + '*.tempo.gt')
beats_filenames = glob.glob(path + '*.beats.gt')

if label_filenames == []:
    label_filenames = np.ones(len(filenames))
if tempi_filenames == []:
    tempi_filenames = np.ones(len(filenames))
if beats_filenames == []:
    beats_filenames = np.ones(len(filenames))

  
# Create an output dictionary:      
titles = [(ele.split('/')[-1]).replace('.wav','') for ele in filenames]
dd = defaultdict(dict)
for title in titles:
    dd[title]['onsets'] = None
    dd[title]['beats'] = None
    dd[title]['tempo'] = None

    
dd_true = copy.deepcopy(dd)

# for loop to iterate all csv files
STFT_torch_list = [] # fill it with all single STFTs
label_torch_list = [] # fill it with all sample labels

for file, label, tempi, beats in zip(filenames, label_filenames, tempi_filenames, beats_filenames):
    sample_rate, data = scipy.io.wavfile.read(file)
    

    detection_func_norm_thres, predicted_onset_arr_time, sample_rate, hop_size = onset_detection(file)

    
    try:
        onset_file_content = pd.read_csv(label,header=None)
        true_onset_arr_time = list(np.array(onset_file_content).reshape((len(onset_file_content ,))))# from shape [x,] we come to shape [1,x]
    except:
        pass
    
       
    
    # Getting title of file:
    title = (file.split('/')[-1]).replace('.wav','') 
    
    # Typing result into dictionary:
    dd[title]['onsets'] = list(predicted_onset_arr_time)
    
    try:
        dd_true[title]['onsets'] = true_onset_arr_time 
    except:
        pass
    
       
    splitter = 7 #int(full_time/6)+1 # each window shall have a size of circa 5 seconds
    found_tempi, tempi_window_sec, start_window_sec, detection_window_,start_window_ind = tempo_detection(detection_func_norm_thres=detection_func_norm_thres,splitter=splitter,hop_size=hop_size,sample_rate=sample_rate)
    beats_sec = beat_detection(start_window_ind=start_window_ind, start_window_sec=start_window_sec,tempo_list_sec=tempi_window_sec,detection_func_norm_thres_wind = detection_window_, hop_size=hop_size, splitter=splitter, sample_rate=sample_rate, predicted_onset_arr_time=predicted_onset_arr_time)
    
    dd[title]['tempo'] = found_tempi#sol#[tempo_found]# [tempo_value1]#[tempo1]# meaning_sol #sol #tempo_list#[tempo2, tempo1]
    dd[title]['beats'] = beats_sec
    
    
    try:
        tempo_file_content1 = pd.read_csv(tempi,header=None,sep='\t',dtype='a')#[0]
        tempo_file_content2 = pd.read_csv(tempi,header=None,sep='\t',dtype='a')#[1]
        true_tempi = [float(tempo_file_content1.values[0][0]),float(tempo_file_content2.values[0][1]),float(tempo_file_content2.values[0][2])]#.reshape((len(tempo_file_content) ,)))
        dd_true[title]['tempo'] = true_tempi
    except:
        dd_true[title]['tempo'] = None
    
    try:
        true_beats = pd.read_csv(beats,header=None,sep='\t')
        true_beats = np.array(true_beats).flatten()
        dd_true[title]['beats'] = list(np.sort(true_beats))
    except:
        dd_true[title]['beats'] = None
    

    

dumped = json.dumps(dd)
with open('final_predictions.json', 'w') as f:
    f.write(dumped + '\n')

dumped_true = json.dumps(dd_true)
with open('groundtruth.json','w') as g:
    g.write(dumped_true)